In [2]:
! pip install --upgrade transformers packaging
import importlib
importlib.reload(__import__('transformers'))

<module 'transformers' from 'c:\\Users\\W-HP\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\transformers\\__init__.py'>

In [7]:
import os
import sys

project_root = os.path.abspath(".")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
%autoreload 2
import importlib

In [9]:
from src.ner.ner_pipeline import (
    load_conll_data,
    print_label_distribution,
    preview_samples,
    create_ner_datasets,
    train_ner_model,
)

ModuleNotFoundError: No module named 'src'

In [ ]:
conll_path = '../data/processed/labeled_data.conll'
all_tokens, all_labels, unique_labels = load_conll_data(conll_path)
print_label_distribution(all_labels)
preview_samples(all_tokens, all_labels, n=3)
print("Unique labels:", unique_labels)

In [ ]:
from transformers import AutoTokenizer

model_name = 'xlm-roberta-base'  # or any model from your list
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset, eval_dataset, label_to_id, id_to_label = create_ner_datasets(
    all_tokens, all_labels, unique_labels, tokenizer,
    max_length=128, test_size=0.2, random_state=42
)
print("Train samples:", len(train_dataset))
print("Eval samples:", len(eval_dataset))

In [ ]:
import transformers
from packaging import version

output_dir = f"models/ner_finetuned/{model_name.replace('/', '_')}"
os.makedirs(output_dir, exist_ok=True)

# Prepare version-compatible kwargs
training_args_kwargs = dict(
    output_dir=output_dir,
    learning_rate=3e-5,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir=os.path.join(output_dir, "logs"),
    logging_steps=50,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none",
    save_total_limit=2,
    seed=42,
)

if version.parse(transformers.__version__) >= version.parse("3.0.0"):
    training_args_kwargs["evaluation_strategy"] = "steps"
else:
    training_args_kwargs["evaluate_during_training"] = True

# Patch train_ner_model to accept **training_args_kwargs
def train_ner_model_version_compatible(*args, **kwargs):
    # Extract training_args_kwargs if present
    training_args_kwargs = kwargs.pop('training_args_kwargs', {})
    from transformers import TrainingArguments
    kwargs['training_args'] = TrainingArguments(**training_args_kwargs)
    from src.ner.ner_pipeline import train_ner_model as original_train_ner_model
    return original_train_ner_model(*args, **kwargs)

# Call the patched function
train_ner_model_version_compatible(
    model_name=model_name,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    label_to_id=label_to_id,
    id_to_label=id_to_label,
    output_dir=output_dir,
    num_train_epochs=5,
    learning_rate=3e-5,
    batch_size=8,
    use_early_stopping=True,
    seed=42,
    training_args_kwargs=training_args_kwargs
)